In [29]:
import json
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torchinfo
from tqdm import tqdm
from timm.models.vision_transformer import VisionTransformer

from local_python.general_utils import (
    load_model,
    load_pd_from_json,
    load_values_from_previous_epochs,
    print_parameters,
    set_seed,
)
from local_python.dataset_util import (
    create_dataloaders,
)

from lora_vit.models import LoRA_ViT_timm
from ssl_library.src.models.fine_tuning.classifiers import LinearClassifier

In [30]:
configuration_csv_path = "./configs/finetune-configuration.csv"
learning_rate = 1e-4

In [31]:
df_config = pd.read_csv(configuration_csv_path)
df_config

,seed,strategy,dataset_path,checkpoint_path
0,1,lora_2,../data_splits/HAM10000_split.csv,../model_weights/vit_t16_v2/ViT_T16-ImageNet_1...
1,1,lora_4,../data_splits/HAM10000_split.csv,../model_weights/vit_t16_v2/ViT_T16-ImageNet_1...


In [1]:
def prepare_model(checkpoint_path, strategy, num_classes, image_shape):
    strategy_params = strategy.split("_")

    if "concat" == strategy_params[0]:
        rank = int(strategy_params[1])
        model = load_model(checkpoint_path, freeze=False, use_ssl_library=True)
        params = list(model.parameters())
        for param in params[: len(params) - rank]:
            param.requires_grad = False
        summary = torchinfo.summary(model, image_shape, batch_dim=0)
        last_output = summary.summary_list[-1].output_size[-1]
        model.head = LinearClassifier(
            last_output,
            num_labels=num_classes,
            use_dropout_in_head=True,
            large_head=False,
            use_bn=True,
        )
    elif "lora" == strategy_params[0]:
        rank = int(strategy_params[1])
        model = load_model(checkpoint_path, freeze=True, use_ssl_library=False)
        summary = torchinfo.summary(model, image_shape, batch_dim=0)
        last_output = summary.summary_list[-1].output_size[-1]
        assert hasattr(model, "blocks"), f"Unknown model type: {type(model)}"
        model.head = LinearClassifier(
            last_output,
            num_labels=num_classes,
            use_dropout_in_head=True,
            large_head=False,
            use_bn=True,
        )
        model = LoRA_ViT_timm(vit_model=model, r=rank, alpha=4)
    else:
        assert False, f"Unknown strategy: {strategy}"

    return model


def train_eval(
    model, optimizer, criterion, start_epoch, end_epoch, dataloaders, loss_file_path, best_loss = None
):
    model = model.to(device)
    for epoch in range(start_epoch, end_epoch):
        model.train()
        print(f"Training epoch {epoch}")
        with open(loss_file_path, "a") as detaillog:
            for i, (images, targets) in enumerate(tqdm(dataloaders["train"])):
                images = images.to(device)
                targets = torch.as_tensor(targets).to(device)

                outputs = model(images)
                loss = criterion(outputs, targets)

                line = {}
                line["epoch"] = epoch
                line["iteration"] = i
                line["loss"] = loss.item()
                line["set"] = "train"
                json.dump(line, detaillog, indent=2)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        torch.save(
            model.state_dict(),
            os.path.join(run_path, f"checkpoint_latest.pth"),
        )

        model.eval()
        valid_loss = []
        with torch.no_grad():
            with open(loss_file_path, "a") as detaillog:
                for i, (images, targets) in enumerate(tqdm(dataloaders["valid"])):
                    images = images.to(device)
                    targets = torch.as_tensor(targets).to(device)

                    outputs = model(images)
                    loss = criterion(outputs, targets)
                    valid_loss.append(loss.item())

                    line = {}
                    line["epoch"] = epoch
                    line["iteration"] = i
                    line["loss"] = valid_loss[-1]
                    line["set"] = "valid"
                    json.dump(line, detaillog, indent=2)
        
        mean_loss = np.array(valid_loss).mean()
        if best_loss is None or mean_loss < best_loss:
            best_loss = mean_loss
            torch.save(
                model.state_dict(),
                os.path.join(run_path, f"checkpoint_best.pth"),
            )


In [33]:
assert torch.cuda.is_available()
n_devices = torch.cuda.device_count()
for i in range(0, n_devices):
    print(torch.cuda.get_device_name(i))

device = torch.device("cuda")

NVIDIA GeForce GTX 960


In [34]:
for _, row in df_config.iterrows():
    seed = row["seed"]
    set_seed(seed)

    checkpoint_path = row["checkpoint_path"]
    model_name = os.path.splitext(os.path.basename(checkpoint_path))[0].replace(
        "_headless", ""
    )

    dataset_path = row["dataset_path"]
    dataset_name = os.path.splitext(os.path.basename(dataset_path))[0].replace(
        "_split", ""
    )

    strategy = row["strategy"]

    run_path = os.path.join("../runs/", dataset_name, model_name, f"{strategy}_{seed}")
    if not os.path.exists(run_path):
        os.makedirs(run_path)
    print(f"Results will be saved to {run_path}")
    

    batch_size = 64
    if strategy.startswith("lora"):
        batch_size = 32

    dataloaders = create_dataloaders(dataset_path, batch_size=batch_size)
    train_class_counts = dataloaders["train"].dataset.get_class_counts()
    print(f"Train class (im)balance: {train_class_counts}")
    num_classes = len(train_class_counts)

    images, _ = next(iter(dataloaders["valid"]))
    image_shape = images.shape[1:]

    model = prepare_model(checkpoint_path, strategy, num_classes, image_shape)
    print_parameters(model)

    class_weights_tensor = torch.tensor(
        1.0 / np.array(list(train_class_counts.values())), dtype=torch.float
    )
    class_weights_tensor = class_weights_tensor.to(device)
    loss_function = nn.CrossEntropyLoss(weight=class_weights_tensor, reduction="mean")
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    (loss_file_path, latest_epoch, best_loss) = load_values_from_previous_epochs(run_path)

    checkpoint_path = os.path.join(run_path, f"checkpoint_latest.pth")
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))
        model.load_state_dict(checkpoint, strict=True)

    checkpoint_path = os.path.join(run_path, f"checkpoint_best.pth")
    if not os.path.exists(checkpoint_path):
        best_loss = None # reset    

    start_epoch = latest_epoch + 1
    end_epoch = start_epoch + 30

    train_eval(
        model,
        optimizer,
        loss_function,
        start_epoch,
        end_epoch,
        dataloaders,
        loss_file_path,
        best_loss = best_loss,
    )

Setting seed to 1
Results will be saved to ../runs/HAM10000\ViT_T16-ImageNet_1k_SSL_Dino\lora_2_1
Set train size: 8908
Set valid size: 1103
Train class (im)balance: {'bkl': 971, 'nv': 5969, 'df': 103, 'mel': 992, 'vasc': 128, 'bcc': 455, 'akiec': 290}
Loading vit_tiny_patch16_224 from timm-library
Ignoring prefix 'model.'
Trainable parameters: 18432/5544583
Read 19171 entries from loss.txt
Latest epoch: 62
Best epoch: 61 with 1.2361385055950709
Training epoch 63


100%|██████████| 35/35 [00:13<00:00,  2.51it/s]


Training epoch 64


100%|██████████| 35/35 [00:13<00:00,  2.51it/s]


Training epoch 65


...

100%|██████████| 35/35 [00:13<00:00,  2.57it/s]


Training epoch 88


100%|██████████| 35/35 [00:13<00:00,  2.53it/s]


Training epoch 89


100%|██████████| 35/35 [00:13<00:00,  2.58it/s]
